In [8]:
import os
import pandas as pd
import numpy as np
import shutil
import sys
import tqdm.notebook as tq
from collections import defaultdict

import torch
import torch.nn as nn

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
df_data = pd.read_csv('MEISD/MEISD_text.csv')

In [11]:
#df_data = df_data.loc[df_data.iloc[:,2:].apply(sum, axis=1) == 1,:].sample(n = 10_000, random_state = 4)
df_data = df_data.sample(n = 10_000, random_state = 4)

In [12]:
df_data

,TV Series,Utterances,dialog_ids,uttr_ids,seasons,episodes,start_times,end_times,sentiment,emotion,intensity,emotion2,intensity2,emotion3,intensity3
2712,CA,that's not all that was long,138,7,3,7,00:05:58:210,00:06:00:010,positive,joy,1,disgust,1,NaN,NaN
18590,TO,or does somebody miss me?,1054,15,3,4,00:02:11:799,00:02:13:265,Positive,surprise,2,like,1,NaN,NaN
19697,TO,"why holly and i should be together,",1110,0,5,26,00:04:09:333,00:04:11:000,positive,joy,2,disgust,1,NaN,NaN
16157,HOC,i'm sorry. i didn't mean for that to come off,931,18,1,4,00:14:58:040,00:14:59:724,negative,surprise,2,disgust,1,NaN,NaN
11635,HMD,"things got a little slow, so we made some wagers",590,17,6,14,00:05:02:407,00:05:05:911,negative,sadness,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13533,BD,how's her driving? driving. is she good behind...,684,18,3,13,00:18:45:626,00:18:49:046,Positive,neutral,NaN,NaN,NaN,NaN,NaN
15237,FR,No-no-no. I don't have a son named Jordie. We ...,841,1,1,23,"00:08:03,566","00:08:10,113",negative,anger,3,NaN,NaN,NaN,NaN
10261,HMD,just because you got out of clinic duty doesn'...,521,14,1,13,00:10:22:227,00:10:24:771,negative,anger,1,disgust,2,NaN,NaN
11528,HMD,yeah. your mother told me,585,8,6,6,00:14:25:386,00:14:27:180,positve,acceptance,2,NaN,NaN,NaN,NaN


In [13]:
import pandas as pd

# Define emotion and sentiment mappings
emotion_map = {
    'neutral': 0,
    'acceptance': 1,
    'disgust': 2,
    'surprise': 3,
    'joy': 4,
    'sadness': 5,
    'anger': 6,
    'like': 7,
    'fear': 8
}

sentiment_map = {
    'positive': 1,
    'negative': 0,
    'neutral': 2
}

# Map and convert sentiment and emotion columns
df_data['sentiment'] = pd.to_numeric(df_data['sentiment'].map(sentiment_map).fillna(2).astype(int))  # Use 2 as a default for unknown
df_data['emotion'] = pd.to_numeric(df_data['emotion'].map(emotion_map).fillna(9).astype(int))  # Use 9 for unknown emotions
df_data['emotion2'] = pd.to_numeric(df_data['emotion2'].map(emotion_map).fillna(9).astype(int))
df_data['emotion3'] = pd.to_numeric(df_data['emotion3'].map(emotion_map).fillna(9).astype(int))

# Convert intensity columns to numeric
df_data['intensity'] = pd.to_numeric(df_data['intensity'], errors='coerce').fillna(0).astype(int)
df_data['intensity2'] = pd.to_numeric(df_data['intensity2'], errors='coerce').fillna(0).astype(int)
df_data['intensity3'] = pd.to_numeric(df_data['intensity3'], errors='coerce').fillna(0).astype(int)


# Convert start and end times to datetime
df_data['start_times'] = pd.to_datetime(df_data['start_times'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_data['end_times'] = pd.to_datetime(df_data['end_times'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df_data.fillna({
    'intensity': 0,
    'intensity2': 0,
    'intensity3': 0,
    'sentiment': 2,  # Default sentiment to neutral
    'emotion': 9,
    'emotion2': 9,
    'emotion3': 9,
    'start_times': pd.NaT,  # Keep NaT for dates
    'end_times': pd.NaT     # Keep NaT for dates
}, inplace=True)


In [14]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_data, random_state=77, test_size=0.30, shuffle=True)
df_test, df_valid = train_test_split(df_test, random_state=88, test_size=0.50, shuffle=True)

In [15]:
print(f"Train: {df_train.shape}, Test: {df_test.shape}, Valid: {df_valid.shape}")

Train: (7000, 15), Test: (1500, 15), Valid: (1500, 15)


In [16]:
# Check data types
print(df_data.dtypes)

# Check unique values in relevant columns
print(df_data['sentiment'].unique())
print(df_data['emotion'].unique())
print(df_data['intensity'].unique())
print(df_data['intensity2'].unique())


TV Series              object
Utterances             object
dialog_ids              int64
uttr_ids                int64
seasons                 int64
episodes                int64
start_times    datetime64[ns]
end_times      datetime64[ns]
sentiment               int32
emotion                 int32
intensity               int32
emotion2                int32
intensity2              int32
emotion3                int32
intensity3              int32
dtype: object
[1 2 0]
[4 3 5 6 0 1 2 9 8 7]
[1 2 0 3]
[1 0 3 2]


In [17]:
# Hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 1e-05

In [18]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

D:\conda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
# Test the tokenizer
test_text = "We are testing BERT tokenizer."
# generate encodings
encodings = tokenizer.encode_plus(test_text,
                                  add_special_tokens = True,
                                  max_length = 50,
                                  truncation = True,
                                  padding = "max_length",
                                  return_attention_mask = True,
                                  return_tensors = "pt")
# we get a dictionary with three keys (see: https://huggingface.co/transformers/glossary.html) 
encodings

{'input_ids': tensor([[  101,  2057,  2024,  5604, 14324, 19204, 17629,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}

In [20]:
df_train['Utterances']

15443                       I thought there wasn't a ball?
13052                        what are you waiting for? go!
4143                             this one time, she was at
7275              while the victor still licks his wounds.
16153    you've begged me to move you up, and now suddenly
                               ...                        
9360     anyway, i told you already, i'm taking a break...
1232     you know what? you sound more and more like mo...
2426                                      at the same time
1237        did you say "ham"? - you're either ham or eggs
6362     Every man who has tasted my cooking has told m...
Name: Utterances, Length: 7000, dtype: object

In [21]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, target_list):
        self.tokenizer = tokenizer
        self.df = df
        self.title = list(df['Utterances'])
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            # Ensure the correct type here
            'targets': torch.FloatTensor(self.targets[index]),
            #'targets': torch.tensor(self.targets[index], dtype=torch.float32),  
            'title': title
        }


In [22]:
target_list = list(df_data.columns)
target_list

['TV Series',
 'Utterances',
 'dialog_ids',
 'uttr_ids',
 'seasons',
 'episodes',
 'start_times',
 'end_times',
 'sentiment',
 'emotion',
 'intensity',
 'emotion2',
 'intensity2',
 'emotion3',
 'intensity3']

In [23]:
target_list = target_list[2:]
target_list = [col for col in target_list if col not in ['start_times', 'end_times']]
target_list

['dialog_ids',
 'uttr_ids',
 'seasons',
 'episodes',
 'sentiment',
 'emotion',
 'intensity',
 'emotion2',
 'intensity2',
 'emotion3',
 'intensity3']

In [24]:
print(df_train[target_list].dtypes)

dialog_ids    int64
uttr_ids      int64
seasons       int64
episodes      int64
sentiment     int32
emotion       int32
intensity     int32
emotion2      int32
intensity2    int32
emotion3      int32
intensity3    int32
dtype: object


In [25]:
train_dataset = CustomDataset(df_train, tokenizer, MAX_LEN, target_list)
valid_dataset = CustomDataset(df_valid, tokenizer, MAX_LEN, target_list)
test_dataset = CustomDataset(df_test, tokenizer, MAX_LEN, target_list)

In [26]:
# Data loaders
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=TRAIN_BATCH_SIZE,
                                                shuffle=True,
                                                num_workers=0
                                                )

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                              batch_size=VALID_BATCH_SIZE,
                                              shuffle=False,
                                              num_workers=0
                                              )

test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=TEST_BATCH_SIZE,
                                               shuffle=False,
                                               num_workers=0
                                               )

In [27]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 11) #numer of classes

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()

# # Freezing BERT layers: (tested, weaker convergence)
# for param in model.bert_model.parameters():
#     param.requires_grad = False

model.to(device)


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [28]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [29]:
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-5)         


D:\conda\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
# Training of the model for one epoch
def train_model(training_loader, model, optimizer):

    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to training mode (activate droput, batch norm)
    model.train()
    # initialize the progress bar
    loop = tq.tqdm(enumerate(training_loader), total=len(training_loader),
                   leave=True, colour='steelblue')
    for batch_idx, data in loop:
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

# forward
        outputs = model(ids, mask, token_type_ids) # (batch,predict)=(32,8)
        loss = loss_fn(outputs, targets)
        losses.append(loss.item())
        # training accuracy
        _, preds = torch.max(outputs, dim=1) # batch dim 
        _, targ = torch.max(targets, dim=1)  # batch dim
        num_samples += len(targ)  # technically adding batch size
        correct_predictions += torch.sum(preds == targ)

        # backward
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

        # Update progress bar
        #loop.set_description(f"")
        #loop.set_postfix(batch_loss=loss)

    # returning: trained model, model accuracy, mean loss
    return model, float(correct_predictions)/num_samples, np.mean(losses)


In [31]:
def eval_model(validation_loader, model, optimizer):
    losses = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            _, preds = torch.max(outputs, dim=1) # batch dim 
            _, targ = torch.max(targets, dim=1)  # batch dim
            num_samples += len(targ)  # technically adding batch size
            correct_predictions += torch.sum(preds == targ)

    return float(correct_predictions)/num_samples, np.mean(losses)
     


In [32]:
history = defaultdict(list)
best_accuracy = 0

for epoch in range(1, EPOCHS+1):
    print(f'Epoch {epoch}/{EPOCHS}')
    model, train_acc, train_loss = train_model(train_data_loader, model, optimizer)
    val_acc, val_loss = eval_model(val_data_loader, model, optimizer)

    print(f'train_loss={train_loss:.4f}, val_loss={val_loss:.4f} train_acc={train_acc:.4f}, val_acc={val_acc:.4f}')

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    # save the best model
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), "best_model_state.bin")
        best_accuracy = val_acc


Epoch 1/3


  0%|          | 0/219 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,7)
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])
plt.grid()

In [ ]:
# Loading pretrained model

model = BERTClass()
model.load_state_dict(torch.load(os.path.join(data_dir,"output","MCTC_model_state.bin")))
model = model.to(device)

In [ ]:
# Evaluate the model using the test data
test_acc, test_loss = eval_model(test_data_loader, model, optimizer)


In [ ]:
# The accuracy looks OK, similar to the validation accuracy
# The model generalizes well !
test_acc
     


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
def get_predictions(model, data_loader):
    model = model.eval()

    titles = []
    predictions = []
    prediction_probs = []
    target_values = []

    with torch.no_grad():
        for data in data_loader:
            title = data["title"]
            ids = data["input_ids"].to(device, dtype = torch.long)
            mask = data["attention_mask"].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data["targets"].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            _, preds = torch.max(outputs, dim=1)
            _, targ = torch.max(targets, dim=1)

            titles.extend(title)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            target_values.extend(targ)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    target_values = torch.stack(target_values).cpu()

    return titles, predictions, prediction_probs, target_values

In [ ]:
titles, predictions, prediction_probs, target_values = get_predictions(model, test_data_loader)


In [ ]:
# Classification report
print(classification_report(target_values, predictions, target_names=target_list))


In [ ]:
import seaborn as sns
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
    plt.ylabel('True category')
    plt.xlabel('Predicted category');

In [ ]:
# raw text
raw_text = "Measurement of D-meson production as a function of charged-particle multiplicity in proton--proton collisions at s√=13 TeV with ALICE at the LHC"


In [ ]:
encoded_text = tokenizer.encode_plus(
    raw_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=True,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)


In [ ]:
input_ids = encoded_text['input_ids'].to(device)
attention_mask = encoded_text['attention_mask'].to(device)
token_type_ids = encoded_text['token_type_ids'].to(device)
output = model(input_ids, attention_mask, token_type_ids)
_, prediction = torch.max(output, dim=1)


In [ ]:
# Correctly predicted the topic of the paper: High energy physics 
print(f'Title: {raw_text}')
print(f'Category: {target_list[prediction]}')
